In [302]:
# 更新版
""" (更新版)
進步中 = true
全局最佳overall_best_obj = -1
已分配設施 = {...都是0}, 蓋廁所也都是0
最後紀錄每次分多少車給哪個j全部加入一個list
config = load_config
while 此輪還在進步 and 還有工廠能蓋
    進步中 = false
    此輪最佳cur_best_obj 暫時=最佳obj（前一輪的）
    紀錄此輪最佳cur_best_config = {}
    紀錄此輪最佳蓋廠點cur_best_j = -1
    對每個可能的建立工廠點 j:
        看看哪個車k對於這個點的V最大 一直塞到此點容納不下車or沒車可塞了，
        剩下用decoration補滿（G最大133.4, E最大100), 此處塞滿, E必滿 記錄塞了多少在廁所在min(M, 缺少的part)
        複製一份config計算總obj (包含先前iter已蓋好之工廠點)
        如果大於此輪最佳，記錄obj, config, j (cur_best)
    如果此輪最佳蓋廠點cur_best_j != -1 代表有找到好的
        進步中=true
        更新全局最佳obj, 全局config, 已分配設施
"""


' (更新版)\n進步中 = true\n全局最佳overall_best_obj = -1\n已分配設施 = {...都是0}, 蓋廁所也都是0\n最後紀錄每次分多少車給哪個j全部加入一個list\nconfig = load_config\nwhile 此輪還在進步 and 還有工廠能蓋\n    進步中 = false\n    此輪最佳cur_best_obj 暫時=最佳obj（前一輪的）\n    紀錄此輪最佳cur_best_config = {}\n    紀錄此輪最佳蓋廠點cur_best_j = -1\n    對每個可能的建立工廠點 j:\n        看看哪個車k對於這個點的V最大 一直塞到此點容納不下車or沒車可塞了，\n        剩下用decoration補滿（G最大133.4, E最大100), 此處塞滿, E必滿 記錄塞了多少在廁所在min(M, 缺少的part)\n        複製一份config計算總obj (包含先前iter已蓋好之工廠點)\n        如果大於此輪最佳，記錄obj, config, j (cur_best)\n    如果此輪最佳蓋廠點cur_best_j != -1 代表有找到好的\n        進步中=true\n        更新全局最佳obj, 全局config, 已分配設施\n'

In [303]:
import sys
sys.path.append('../')
import utility as util
import copy
import math

E_MAX_INPUT = 100
G_MAX_INPUT = 133
# config = util.load_specific_yaml('instance_F_1.yaml')
# config['A_EX_bound'] = config['M']
# print(config)
config = util.load_specific_yaml('./bingray_instance0429/instance_S_1.yaml')
def G_function(x):
  if x > G_MAX_INPUT:
    return 1
  else:
    return -0.000015 * (x**2) + 0.0095 * x
def E_function(x):
	if x > E_MAX_INPUT:
		return 40
	else:
	  return -0.004 * (x**2) + 0.8 * x


In [304]:
def greedy_best_location(iter_config: dict, candidates: list, compensate_attractiveness: list, cars_usage_record: list, total_util_list: list):
  '''
  用V最大的車子不斷填入直到某個點不能再承受更多車(效用E堆滿了 E_MAX_INPUT) 
  輸入：暫時複製的config, 有無蓋設施list, 補償蓋廁所總表, 車子使用總紀錄
  輸出：回傳新的config應該長怎樣（如果沒有更好的obj那新的config就不會更新到全局), 能找到最好的新obj, 建立哪個j, 蓋多少廁所在這個點, 用了哪些車給誰的總表包含過往紀錄 [(1, 5, 18),(0, 4, 18),...] (第k種車, 幾輛, 給哪個j) , 這些蓋的點最後的總utility之list
  best_iter_config, best_obj, best_loc, best_compensate_attractiveness, best_cars_usage_record, best_total_util_list
  '''
  best_iter_config, best_obj, best_loc, best_compensate_attractiveness, best_cars_usage_record, best_total_util_list = iter_config, -1, -1, None, None, None
  # Find the best location with most obj
  for ind, facility in enumerate(candidates):
    if facility == 0:  # haven't built this j, start filling with largest V 
      car_list = [(value, index) for index, value in enumerate(iter_config['V'][ind])]
      sorted_car_list = sorted(car_list, reverse=True)  # [(488183, 2), (420751, 0), (133684, 1)]
      quota_loc = copy.deepcopy(iter_config['U_L'][ind])  # U_L : Max capacity for facility j, Dim : (j) # The total resource allowed to be used in this location.
      quota_loc_k = copy.deepcopy(iter_config['U_LT'][ind])  # U_LT : Max amount of resource k allowed to be allocated to j, Dim : (j, k) #specific resource allowed to be used in this location.
      quota_k = copy.deepcopy(iter_config['U_T'])  # U_T : Max allocation limit for resource k, Dim : (k) # Max allocation limit for resource k

      cur_to_fill = []
      cur_util = 0
      while quota_loc > 0:  # 一種一種車去填依照效用順序
        if(len(sorted_car_list) == 0): break  # all resources are used up in this location.
        elif(cur_util >= E_MAX_INPUT): break  # 此點效益 E 已滿
        car_to_fill = sorted_car_list.pop(0)  # (488183, 2)
				# The number allowed to fill is the minimum of these 3+1 limitations. 第四個是剛好填滿他就好用越少的這種車
        num_to_fill = min(quota_loc, quota_loc_k[car_to_fill[1]], quota_k[car_to_fill[1]], math.ceil((E_MAX_INPUT-cur_util) / car_to_fill[0]))
        quota_k[car_to_fill[1]] -= num_to_fill
        quota_loc_k[car_to_fill[1]] -= num_to_fill
        quota_loc -= num_to_fill
        cur_util += num_to_fill * car_to_fill[0]
        cur_to_fill.append((car_to_fill[1], num_to_fill))  # Record the (car index , and the number of cars) to fill.

      #TODO calculate total obj using copy config
      cur_config = copy.deepcopy(iter_config)
      cur_config = update_config_each_iteration_build_j(cur_config, cur_to_fill)
      tmp_candidates = copy.deepcopy(candidates)
      tmp_candidates[ind] = 1
      tmp_compensate_attractiveness = copy.deepcopy(compensate_attractiveness)
      tmp_compensate_attractiveness[ind] = min(max(0, E_MAX_INPUT - cur_util), config['A_EX_bound'])  #cur_util有可能超過E_MAX_INPUT(100)在塞某台車後
      print(f"這次的cur_util:{cur_util}, 蓋多少廁所？{tmp_compensate_attractiveness[ind]}")
      print("compensate:", tmp_compensate_attractiveness)
      tmp_cars_usage_record = copy.deepcopy(cars_usage_record)
      tmp_cars_usage_record.extend([(x[0], x[1], ind) for x in cur_to_fill])  # 紀錄進總共分配掉的車子, 額外紀錄分配給哪個j
      tmp_total_util_list = copy.deepcopy(total_util_list) 
      tmp_total_util_list[ind] = cur_util + tmp_compensate_attractiveness[ind]
      #TODO compensate_atteactiveness, cars_usage_record
      cur_obj = calc_current_gain(cur_config, tmp_candidates, tmp_total_util_list) - calc_current_cost(cur_config, tmp_candidates, tmp_compensate_attractiveness, tmp_cars_usage_record)
      if (cur_obj > best_obj):
        best_obj = cur_obj
        best_loc = ind
        best_compensate_attractiveness = copy.deepcopy(tmp_compensate_attractiveness)
        print("compensate:", tmp_compensate_attractiveness)
        print("bestcompensate:", best_compensate_attractiveness)
        best_cars_usage_record = copy.deepcopy(tmp_cars_usage_record)
        best_iter_config = cur_config
        best_total_util_list = copy.deepcopy(tmp_total_util_list)
        print("New best location found!")
  print(f"Iteration ended! Find the best location: {best_loc}")
  print(f"Best obj: {best_obj}")
  return best_iter_config, best_obj, best_loc, best_compensate_attractiveness, best_cars_usage_record, best_total_util_list


def update_config_each_iteration_build_j(config: dict, cars_to_fill: list):  # 把用掉的車子扣掉
  for (car_type, allocate_num) in cars_to_fill:
    # print(f"One combination:{car_type}, {allocate_num} (update_config func)")
    config['U_T'][car_type] -= allocate_num
  return config

def calc_current_gain(config: dict, facility_is_built: list, total_util_list: list):  
  total_gain = 0
  for customer_pt_i in range(config["i_amount"]):
    total_attr_i, our_vs_all_percentage = calc_total_attr_i(config, customer_pt_i, facility_is_built, total_util_list)
    total_gain += config['H'][customer_pt_i] * G_function(total_attr_i) * our_vs_all_percentage
    print(f"第{customer_pt_i}個顧客點, total_attr={total_attr_i}, G={G_function(total_attr_i)}, 我們取得的比例={our_vs_all_percentage}, 賺到多少錢={config['H'][customer_pt_i] * G_function(total_attr_i) * our_vs_all_percentage}")
  print(f"總共賺到多少錢?={total_gain}")
  return total_gain

def calc_total_attr_i(config: dict, customer_pt_i: int, facility_is_built: list, total_util_list: list):
  """
  assume if loc j is built, its E is 40 -> modify: through E's func
  ===
  parameters: 
  - customer_pt_i, represent the customer_point i
  - facility_is_built, looks like [0, 1, 0, 0, 1, 1, 1, 1]
  - total_util_list, looks like [0, 0, 98, 101, 33, 0, 100]
  returns:
  - TAi, total attractiveness for this point
  - our_vs_all_percentage, 對於這個顧客i點的貢獻, 我們廠vs全部的比例（P）
  """
  total_attr_i = 0
  # first calculate all our facility contribute to this i
  for j_idx, facility in enumerate(facility_is_built):
    if facility == 1:
      total_attr_i += E_function(total_util_list[j_idx]) / (config['D'][customer_pt_i][j_idx] ** 2) 
  only_our_facilities_total_attr = total_attr_i  # 暫時結算 只有我們設施的吸引力 之後拿來算比例
  # add opponent's attr contribute to this i
  for l_idx in range(config['l_amount']):
    total_attr_i += config['A_opponent_bar'][l_idx] / (config['D_comp'][customer_pt_i][l_idx] ** 2)
  return total_attr_i, only_our_facilities_total_attr / total_attr_i


def calc_current_cost(config: dict, facility_is_built: list, compensate_attractiveness: list, cars_usage_record: list):
  total_cost = 0
  build_cost = sum(x * y for x, y in zip(facility_is_built, config['F']))
  attr_cost = sum(x * y for x, y in zip(compensate_attractiveness, config['C']))
  total_cost = total_cost + build_cost + attr_cost
  for (car_type, allocate_num, loc_j) in cars_usage_record:
    total_cost += config['B'][loc_j][car_type] * allocate_num
  print(f"建造花費：{build_cost}, 廁所花費：{attr_cost}, 車子花費：{total_cost-build_cost-attr_cost}, 總花費：{total_cost}\n")
  return total_cost

In [305]:
improve = True
overall_best_obj = 0
candidates = [0 for i in range(config['j_amount'])]
compensate_attractiveness = [0 for i in range(config['j_amount'])]   # addition attr to fulfill e 
total_util_list = [0 for i in range(config['j_amount'])]  # 紀錄每個點的總attr (車子+廁所)
cars_usage_record = []  # 紀錄每次分了多少車給哪個j [(1, 5, 18),(0, 4, 18),(2, 3, 18),(2, 5, 2),...] (第k種車, 幾輛, 給哪個j)
while (improve and any(element != 1 for element in candidates)):  #每輪多建一個點
    print("下一輪開始")
    print(compensate_attractiveness)
    improve = False
    iter_best_config = copy.deepcopy(config)
    cur_config, cur_obj, cur_loc_to_build, cur_compensate_attractiveness, cur_cars_usage_record, cur_total_util_list = greedy_best_location(iter_best_config, candidates, compensate_attractiveness, cars_usage_record, total_util_list)
    print(cur_compensate_attractiveness)
    print(f"current obj: {cur_obj}, loc_to_build: {cur_loc_to_build}")
    if cur_obj > overall_best_obj:
        improve = True
        config.update(cur_config)  #不確定update能不能，不能就改deepcopy!!!!!!!!!!!!!!
        overall_best_obj = cur_obj
        candidates[cur_loc_to_build] = 1
        compensate_attractiveness = copy.deepcopy(cur_compensate_attractiveness)
        cars_usage_record = copy.deepcopy(cur_cars_usage_record)
        total_util_list = copy.deepcopy(cur_total_util_list)
    print("\n\n這輪的結果：")
    print("建築蓋了沒的list：", candidates)
    print("車子使用紀錄：", cars_usage_record)
    print("蓋廁所總表：", compensate_attractiveness)
    print("效用總表對於每個點：", total_util_list)
    
print(f"最終結果overall_best_obj:{overall_best_obj}")



下一輪開始
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
這次的cur_util:18, 蓋多少廁所？34
compensate: [34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
第0個顧客點, total_attr=3.821625, G=0.036086365235390626, 我們取得的比例=0.12586268930101724, 賺到多少錢=2.7251561853750004
第1個顧客點, total_attr=7.326714482758621, G=0.06879857640952891, 我們取得的比例=0.08403220863169038, 賺到多少錢=3.277995017073804
第2個顧客點, total_attr=23.81032459660206, G=0.21769411030678626, 我們取得的比例=0.012197023739066008, 賺到多少錢=1.8135154179551738
總共賺到多少錢?=7.8166666204039785
建造花費：9, 廁所花費：34, 車子花費：54, 總花費：97

這次的cur_util:36, 蓋多少廁所？34
compensate: [0, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
第0個顧客點, total_attr=3.839255136986301, G=0.03625182560126672, 我們取得的比例=0.12987679099068966, 賺到多少錢=2.82496246598799
第1個顧客點, total_attr=8.852210953346857, G=0.08292057947535683, 我們取得的比例=0.24188041630195178, 賺到多少錢=11.372242048743587
第2個顧客點, total_attr=24.11663081373785, G=0.2203838145034176, 我們取

In [306]:
"""
這次的cur_util:80, 蓋多少廁所？20
第0個顧客點, total_attr=43.340625, 我們取得的比例=0.9229216237652318, 賺到多少錢=212.397375
第1個顧客點, total_attr=7.065016783643577, 我們取得的比例=0.05010353290376752, 賺到多少錢=1.8854555866389773
第2個顧客點, total_attr=23.932280636283057, 我們取得的比例=0.017230749559046802, 賺到多少錢=2.5745623855180537
總共賺到多少錢?=216.85739297215704
建造花費：9, 廁所花費：40, 車子花費：24, 總花費：73
"""

'\n這次的cur_util:80, 蓋多少廁所？20\n第0個顧客點, total_attr=43.340625, 我們取得的比例=0.9229216237652318, 賺到多少錢=212.397375\n第1個顧客點, total_attr=7.065016783643577, 我們取得的比例=0.05010353290376752, 賺到多少錢=1.8854555866389773\n第2個顧客點, total_attr=23.932280636283057, 我們取得的比例=0.017230749559046802, 賺到多少錢=2.5745623855180537\n總共賺到多少錢?=216.85739297215704\n建造花費：9, 廁所花費：40, 車子花費：24, 總花費：73\n'

In [307]:
cars_usage_record

[(0, 8, 15), (1, 6, 13), (0, 7, 13), (1, 9, 2), (0, 8, 2)]

In [308]:
config

{'A_EX_bound': 34,
 'A_opponent_bar': [36, 41, 25, 9],
 'B': [[4, 5],
  [3, 3],
  [5, 3],
  [3, 4],
  [5, 5],
  [4, 5],
  [3, 3],
  [5, 5],
  [5, 4],
  [3, 5],
  [4, 3],
  [4, 3],
  [4, 3],
  [4, 4],
  [3, 3],
  [3, 3],
  [3, 4],
  [3, 5],
  [5, 4],
  [3, 3],
  [3, 4],
  [5, 3],
  [3, 3],
  [4, 4],
  [4, 3]],
 'C': [1,
  1,
  1,
  2,
  2,
  3,
  1,
  3,
  3,
  2,
  1,
  3,
  1,
  3,
  2,
  2,
  2,
  3,
  1,
  2,
  1,
  2,
  3,
  3,
  2],
 'CANDIDATES': [[0, 0],
  [0, 3],
  [0, 6],
  [0, 9],
  [0, 12],
  [3, 0],
  [3, 3],
  [3, 6],
  [3, 9],
  [3, 12],
  [6, 0],
  [6, 3],
  [6, 6],
  [6, 9],
  [6, 12],
  [9, 0],
  [9, 3],
  [9, 6],
  [9, 9],
  [9, 12],
  [12, 0],
  [12, 3],
  [12, 6],
  [12, 9],
  [12, 12]],
 'COMPETITORS': [[4, 4], [4, 8], [8, 4], [8, 8]],
 'CONSUMERS': [[8, 0], [1, 7], [5, 9]],
 'D': [[8.0,
   8.54400374531753,
   10.0,
   12.041594578792296,
   14.422205101855956,
   5.0,
   5.830951894845301,
   7.810249675906654,
   10.295630140987,
   13.0,
   2.0,
   3.6055512754

In [309]:
candidates

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [310]:
cars_usage_record  # (第k種車, 幾輛, 給哪個j)

[(0, 8, 15), (1, 6, 13), (0, 7, 13), (1, 9, 2), (0, 8, 2)]

In [311]:
compensate_attractiveness

[0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [312]:
total_util_list

[0,
 0,
 100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 60,
 0,
 100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]